# 時中指數

# Load Data 

In [25]:
import pandas as pd
from datetime import datetime, timedelta

df = pd.read_csv('D:\\project_code\\bigdata_class_v1\\resources\\w7\\w07-1-Voice of share on Chen時中聲量\\t\\cna_news_preprocessed.csv',sep='|')

In [26]:
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,index.php?k=1_2025-03-27_1,2025-03-27,PC,多平台\n《餓狼傳說 City of the Wolves》足球界傳奇巨星「克里斯蒂亞諾‧羅...,株式會社 SNK 今日（3/27）宣布，足球界傳奇巨星「克里斯蒂亞諾‧羅納度」確定將以可遊玩...,暫無,暫無,"[('遊戲', 8), ('足球', 8), ('傳說', 7), ('角色', 6), (...","['株式會社', ' SNK ', '今日', '（', '3/27', '）', '宣布'...","['株式會社', '宣布', '足球界', '傳奇', '巨星', '克里斯蒂亞諾‧羅納度'...","[NerToken(word='株式會社', ner='ORG', idx=(0, 4)),...","[('株式會社', 'Nc'), (' SNK ', 'FW'), ('今日', 'Nd')...",https://gnn.gamer.com.tw/detail.php?sn=282906,https://p2.bahamut.com.tw/B/2KU/58/2fc0d3915aa...


In [27]:
df.shape

(391, 14)

# Filter news for selected keywords

In [28]:
import pandas as pd
from datetime import datetime, timedelta
import re

# # Load Data 
# df = pd.read_csv('./news_dataset_preprocessed_for_django.csv',sep='|')
# Step 0: Filter news articles using the following function
# Searching keywords from "content" column
def filter_df_via_content(df, query_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date_delta = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')
    start_date_min = df.date.min()
    # set start_date as the larger one from the start_date_delta and start_date_min 開始時間選資料最早時間與周數:兩者較晚者
    start_date = max(start_date_delta,   start_date_min)


    # (1) proceed filtering: a duration of a period of time
    # 期間條件
    period_condition = (df.date >= start_date) & (df.date <= end_date) 
    
    # (2) proceed filtering: news category
    # 新聞類別條件
    if (cate == "全部"):
        condition = period_condition  # "全部"類別不必過濾新聞種類
    else:
        # 過濾category新聞類別條件
        condition = period_condition & (df.category == cate)

    # (3) proceed filtering: and or
    # and or 條件
    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.content.apply(lambda text: all((qk in text) for qk in query_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.content.apply(lambda text: isinstance(text, str) and any(qk in text for qk in query_keywords))
 #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query


In [29]:
query_keywords = ['英雄聯盟','lol'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
weeks=4
cond='or'
cate='全部'
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)

In [30]:
len(df_query)

13

# Count how many pieces of news containing these keywords計算各類別多少篇文章提到該關鍵字

# Count how many times these keywords were mentioned in each category計算各類別出現關鍵字次數

In [31]:
# **計算各類別多少篇文章提到該關鍵字
# **計算各類別出現關鍵字次數

news_categories = ['PC','動漫畫','電競','活動展覽']

def count_keyword(df_query, query_keywords):
    cate_occurrence = {}
    cate_freq = {}

    # 字典初始化
    for cate in news_categories:
        cate_occurrence[cate] = 0
        cate_freq[cate] = 0

    # 初始化 '全部'
    cate_occurrence['全部'] = 0
    cate_freq['全部'] = 0

    for idx, row in df_query.iterrows():
        # count the number of articles 各類別篇數統計
        cate_occurrence[row.category] += 1
        cate_occurrence['全部'] += 1
        
        # count the keyword frequency 各類別次數統計
        freq = sum([len(re.findall(keyword, row.content, re.I)) for keyword in query_keywords])
        cate_freq[row.category] += freq
        cate_freq['全部'] += freq

    total_articles = cate_occurrence['全部']
    total_frequency = cate_freq['全部']
    return cate_freq, cate_occurrence, total_articles, total_frequency


In [32]:
query_keywords = ['英雄聯盟','lol'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
weeks=4
cond='or'
cate='全部'
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
count_keyword(df_query, query_keywords)

({'PC': 4, '動漫畫': 0, '電競': 20, '活動展覽': 6, '全部': 30},
 {'PC': 1, '動漫畫': 0, '電競': 11, '活動展覽': 1, '全部': 13},
 13,
 30)

In [33]:
query_keywords = ['英雄聯盟','lol'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
weeks=4
cond='or'
cate='全部'
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
count_keyword(df_query, query_keywords)

({'PC': 4, '動漫畫': 0, '電競': 20, '活動展覽': 6, '全部': 30},
 {'PC': 1, '動漫畫': 0, '電競': 11, '活動展覽': 1, '全部': 13},
 13,
 30)

In [34]:
import re

In [35]:
content="陳時中,....，陳時中00，陳時中說，"
keyword="陳時中"
re.findall(keyword, content)

['陳時中', '陳時中', '陳時中']

In [36]:
len(re.findall(keyword, content))

3

In [37]:
keyword="時中"
re.findall(keyword, content)

['時中', '時中', '時中']

In [38]:
keyword="陳中"
re.findall(keyword, content)

[]

# Caclulate date-based reported frequency of these keywords計算被報導的次數以時間為基礎

In [39]:
def get_keyword_occurrence_time_series(df_query):
    date_samples = df_query.date
    query_freq = pd.DataFrame({'date_index': pd.to_datetime(date_samples), 'freq': [1 for _ in range(len(df_query))]})
    data = query_freq.groupby(pd.Grouper(key='date_index', freq='D')).sum()
    line_xy_data = []
    for i, idx in enumerate(data.index):
        row = {'x': idx.strftime('%Y-%m-%d'), 'y': int(data.iloc[i].freq)}
        line_xy_data.append(row)
    return line_xy_data

In [40]:
get_keyword_occurrence_time_series(df_query)

[{'x': '2025-03-03', 'y': 1},
 {'x': '2025-03-04', 'y': 0},
 {'x': '2025-03-05', 'y': 1},
 {'x': '2025-03-06', 'y': 0},
 {'x': '2025-03-07', 'y': 0},
 {'x': '2025-03-08', 'y': 0},
 {'x': '2025-03-09', 'y': 0},
 {'x': '2025-03-10', 'y': 2},
 {'x': '2025-03-11', 'y': 1},
 {'x': '2025-03-12', 'y': 1},
 {'x': '2025-03-13', 'y': 1},
 {'x': '2025-03-14', 'y': 0},
 {'x': '2025-03-15', 'y': 2},
 {'x': '2025-03-16', 'y': 0},
 {'x': '2025-03-17', 'y': 2},
 {'x': '2025-03-18', 'y': 0},
 {'x': '2025-03-19', 'y': 0},
 {'x': '2025-03-20', 'y': 0},
 {'x': '2025-03-21', 'y': 1},
 {'x': '2025-03-22', 'y': 0},
 {'x': '2025-03-23', 'y': 0},
 {'x': '2025-03-24', 'y': 0},
 {'x': '2025-03-25', 'y': 1}]

# Put them together

In [41]:

query_keywords = ['英雄聯盟','lol'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
weeks=1
cond='or'
cate='全部'

df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
cate_freq, cate_occurrence, total_articles, total_frequency = count_keyword(df_query, query_keywords)
freqByDate = get_keyword_occurrence_time_series(df_query)


selectedCategories = ['PC','動漫畫','電競','活動展覽']
# selectedCategories = ['政治', '產經', '生活', '國際', '社會', '兩岸','全部']

freqByCate = [cate_occurrence[k] for k in selectedCategories]

response =  {'freqByDate': freqByDate, # 時間序列資料
           'freqByCate': freqByCate, # 各類別的篇數
           'category': selectedCategories, # 類別名稱
           'num_frequency': cate_freq['全部'], # 這關鍵字被提多少次
           'num_occurrence': cate_occurrence['全部'] #多少篇提到這關鍵字
           }

In [42]:
freqByCate

[1, 0, 1, 0]

In [43]:
cate_freq

{'PC': 4, '動漫畫': 0, '電競': 4, '活動展覽': 0, '全部': 8}

In [44]:
cate_occurrence

{'PC': 1, '動漫畫': 0, '電競': 1, '活動展覽': 0, '全部': 2}

In [45]:
freqByDate

[{'x': '2025-03-21', 'y': 1},
 {'x': '2025-03-22', 'y': 0},
 {'x': '2025-03-23', 'y': 0},
 {'x': '2025-03-24', 'y': 0},
 {'x': '2025-03-25', 'y': 1}]

In [46]:
response

{'freqByDate': [{'x': '2025-03-21', 'y': 1},
  {'x': '2025-03-22', 'y': 0},
  {'x': '2025-03-23', 'y': 0},
  {'x': '2025-03-24', 'y': 0},
  {'x': '2025-03-25', 'y': 1}],
 'freqByCate': [1, 0, 1, 0],
 'category': ['PC', '動漫畫', '電競', '活動展覽'],
 'num_frequency': 8,
 'num_occurrence': 2}

In [47]:
response.items()

dict_items([('freqByDate', [{'x': '2025-03-21', 'y': 1}, {'x': '2025-03-22', 'y': 0}, {'x': '2025-03-23', 'y': 0}, {'x': '2025-03-24', 'y': 0}, {'x': '2025-03-25', 'y': 1}]), ('freqByCate', [1, 0, 1, 0]), ('category', ['PC', '動漫畫', '電競', '活動展覽']), ('num_frequency', 8), ('num_occurrence', 2)])

In [48]:
list(response.items())

[('freqByDate',
  [{'x': '2025-03-21', 'y': 1},
   {'x': '2025-03-22', 'y': 0},
   {'x': '2025-03-23', 'y': 0},
   {'x': '2025-03-24', 'y': 0},
   {'x': '2025-03-25', 'y': 1}]),
 ('freqByCate', [1, 0, 1, 0]),
 ('category', ['PC', '動漫畫', '電競', '活動展覽']),
 ('num_frequency', 8),
 ('num_occurrence', 2)]

## Save data to csv file

In [49]:
df_data = pd.DataFrame(list(response.items()),columns=['name','value'])

In [50]:
df_data

,name,value
0,freqByDate,"[{'x': '2025-03-21', 'y': 1}, {'x': '2025-03-2..."
1,freqByCate,"[1, 0, 1, 0]"
2,category,"[PC, 動漫畫, 電競, 活動展覽]"
3,num_frequency,8
4,num_occurrence,2


In [51]:
## 存成csv格式檔案
df_data.to_csv('lol.csv',sep=',', index=None)

### Alternative way: using zip

In [52]:
k=list(response.keys())
v=list(response.values())

In [53]:
#list(zip(k,v))

In [54]:
df_data = pd.DataFrame(list(zip(k,v)),columns=['name','value'])
df_data

,name,value
0,freqByDate,"[{'x': '2025-03-21', 'y': 1}, {'x': '2025-03-2..."
1,freqByCate,"[1, 0, 1, 0]"
2,category,"[PC, 動漫畫, 電競, 活動展覽]"
3,num_frequency,8
4,num_occurrence,2


## Read csv file and convert to dict format

In [55]:
df_data = pd.read_csv('lol.csv')

In [56]:
df_data 

,name,value
0,freqByDate,"[{'x': '2025-03-21', 'y': 1}, {'x': '2025-03-2..."
1,freqByCate,"[1, 0, 1, 0]"
2,category,"['PC', '動漫畫', '電競', '活動展覽']"
3,num_frequency,8
4,num_occurrence,2


In [57]:
# Convert to dictionary format
dict(list(df_data.values))

{'freqByDate': "[{'x': '2025-03-21', 'y': 1}, {'x': '2025-03-22', 'y': 0}, {'x': '2025-03-23', 'y': 0}, {'x': '2025-03-24', 'y': 0}, {'x': '2025-03-25', 'y': 1}]",
 'freqByCate': '[1, 0, 1, 0]',
 'category': "['PC', '動漫畫', '電競', '活動展覽']",
 'num_frequency': '8',
 'num_occurrence': '2'}

In [58]:
list(df_data.values)

[array(['freqByDate',
        "[{'x': '2025-03-21', 'y': 1}, {'x': '2025-03-22', 'y': 0}, {'x': '2025-03-23', 'y': 0}, {'x': '2025-03-24', 'y': 0}, {'x': '2025-03-25', 'y': 1}]"],
       dtype=object),
 array(['freqByCate', '[1, 0, 1, 0]'], dtype=object),
 array(['category', "['PC', '動漫畫', '電競', '活動展覽']"], dtype=object),
 array(['num_frequency', '8'], dtype=object),
 array(['num_occurrence', '2'], dtype=object)]

In [59]:
response = dict(list(df_data.values))


In [60]:
response['num_occurrence']

'2'

In [61]:
type(response['num_occurrence'])


str

In [62]:
type(response['freqByCate'])


str

In [63]:
response['freqByCate'] = eval(response['freqByCate'])


In [64]:
response


{'freqByDate': "[{'x': '2025-03-21', 'y': 1}, {'x': '2025-03-22', 'y': 0}, {'x': '2025-03-23', 'y': 0}, {'x': '2025-03-24', 'y': 0}, {'x': '2025-03-25', 'y': 1}]",
 'freqByCate': [1, 0, 1, 0],
 'category': "['PC', '動漫畫', '電競', '活動展覽']",
 'num_frequency': '8',
 'num_occurrence': '2'}

### # How to convert a list into a dict?

In [65]:
# How to convert a list into a dict?
[['one',[1,2,3]], ['two',2]]

[['one', [1, 2, 3]], ['two', 2]]

In [66]:
# How to convert a list into a dict?
[['one',[1,2,3]], ['two',2]]

[['one', [1, 2, 3]], ['two', 2]]

In [67]:
dict([['one',[1,2,3]], ['two',2]])

{'one': [1, 2, 3], 'two': 2}

## All-in-one function

In [70]:
# Load Data 
import pandas as pd
from datetime import datetime, timedelta

df = pd.read_csv('D:\\project_code\\bigdata_class_v1\\resources\\w7\\w07-1-Voice of share on Chen時中聲量\\t\\cna_news_preprocessed.csv',sep='|')

In [71]:


def process_data_all_in_one(query_keywords, weeks):


    cond='or'
    cate='全部'

    df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
    cate_freq, cate_occurrence, _, _ = count_keyword(df_query, query_keywords)
    freqByDate = get_keyword_occurrence_time_series(df_query)


    selectedCategories = ['PC','動漫畫','電競','活動展覽']
    # selectedCategories = ['全部', '政治', '產經', '生活', '社會']
    # selectedCategories = ['全部', '政治', '兩岸', '產經', '生活', '社會']
    # selectedCategories = ['政治', '產經', '生活', '國際', '社會', '兩岸','全部']

    freqByCate = [cate_occurrence[k] for k in selectedCategories]

    response =  {'freqByDate': freqByDate,
            'freqByCate': freqByCate,
            'category': selectedCategories,
            'num_frequency': cate_freq['全部'], # 這關鍵字被提多少次
            'num_occurrence': cate_occurrence['全部'] #多少篇提到這關鍵字
            }

    

    return response

In [72]:
query_keywords = ['英雄聯盟','lol'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
weeks=4
data_response = process_data_all_in_one(query_keywords, weeks)

In [73]:
df_data = pd.DataFrame(list(data_response.items()),columns=['name','value'])
## 存成csv格式檔案
df_data.to_csv('chen_shih_chung_data.csv',sep=',', index=None)

In [74]:
df_data = pd.read_csv('lol.csv')

In [75]:
df_data 

,name,value
0,freqByDate,"[{'x': '2025-03-21', 'y': 1}, {'x': '2025-03-2..."
1,freqByCate,"[1, 0, 1, 0]"
2,category,"['PC', '動漫畫', '電競', '活動展覽']"
3,num_frequency,8
4,num_occurrence,2


# This is your featured app

In [ ]:
#query_keywords = ['蔡英文','小英'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
#query_keywords = ['賴清德'] #不要有子字串否則會重複計算 
#query_keywords = ['柯文哲','柯P','柯p'] #不要有子字串否則會重複計算 
query_keywords = ['蘋果','Apple','ipad','iphone','imac'] #不要有子字串否則會重複計算 
weeks=4
data_response = process_data_all_in_one(query_keywords, weeks)

In [ ]:
data_response

{'freqByDate': [{'x': '2022-03-27', 'y': 1},
  {'x': '2022-03-28', 'y': 6},
  {'x': '2022-03-29', 'y': 9},
  {'x': '2022-03-30', 'y': 6},
  {'x': '2022-03-31', 'y': 6},
  {'x': '2022-04-01', 'y': 7},
  {'x': '2022-04-02', 'y': 2},
  {'x': '2022-04-03', 'y': 1},
  {'x': '2022-04-04', 'y': 1},
  {'x': '2022-04-05', 'y': 0},
  {'x': '2022-04-06', 'y': 5},
  {'x': '2022-04-07', 'y': 1},
  {'x': '2022-04-08', 'y': 3},
  {'x': '2022-04-09', 'y': 1},
  {'x': '2022-04-10', 'y': 0},
  {'x': '2022-04-11', 'y': 5},
  {'x': '2022-04-12', 'y': 5},
  {'x': '2022-04-13', 'y': 4},
  {'x': '2022-04-14', 'y': 2},
  {'x': '2022-04-15', 'y': 5},
  {'x': '2022-04-16', 'y': 2},
  {'x': '2022-04-17', 'y': 3},
  {'x': '2022-04-18', 'y': 2},
  {'x': '2022-04-19', 'y': 4},
  {'x': '2022-04-20', 'y': 5},
  {'x': '2022-04-21', 'y': 4},
  {'x': '2022-04-22', 'y': 3},
  {'x': '2022-04-23', 'y': 5}],
 'freqByCate': [98, 0, 8, 4],
 'category': ['全部', '政治', '生活', '社會'],
 'num_frequency': 409,
 'num_occurrence': 98}

In [ ]:
query_keywords = ['疫情','新冠'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
weeks=4
data_response = process_data_all_in_one(query_keywords, weeks)

In [ ]:
data_response

{'freqByDate': [{'x': '2022-03-26', 'y': 46},
  {'x': '2022-03-27', 'y': 60},
  {'x': '2022-03-28', 'y': 99},
  {'x': '2022-03-29', 'y': 74},
  {'x': '2022-03-30', 'y': 104},
  {'x': '2022-03-31', 'y': 98},
  {'x': '2022-04-01', 'y': 75},
  {'x': '2022-04-02', 'y': 52},
  {'x': '2022-04-03', 'y': 57},
  {'x': '2022-04-04', 'y': 64},
  {'x': '2022-04-05', 'y': 70},
  {'x': '2022-04-06', 'y': 97},
  {'x': '2022-04-07', 'y': 119},
  {'x': '2022-04-08', 'y': 98},
  {'x': '2022-04-09', 'y': 74},
  {'x': '2022-04-10', 'y': 57},
  {'x': '2022-04-11', 'y': 103},
  {'x': '2022-04-12', 'y': 120},
  {'x': '2022-04-13', 'y': 150},
  {'x': '2022-04-14', 'y': 105},
  {'x': '2022-04-15', 'y': 94},
  {'x': '2022-04-16', 'y': 69},
  {'x': '2022-04-17', 'y': 71},
  {'x': '2022-04-18', 'y': 101},
  {'x': '2022-04-19', 'y': 122},
  {'x': '2022-04-20', 'y': 114},
  {'x': '2022-04-21', 'y': 116},
  {'x': '2022-04-22', 'y': 87},
  {'x': '2022-04-23', 'y': 56}],
 'freqByCate': [2552, 210, 1023, 38],
 'categor

# views.py in Django

In [ ]:
from django.http import JsonResponse
from django.shortcuts import render
import pandas as pd

def load_data_scchen():
    # Read data from csv file
    df_data = pd.read_csv('app_scchen/dataset/chen_shih_chung_data.csv',sep=',')
    global response
    response = dict(list(df_data.values))
    del df_data

# load pk data
load_data_scchen()

def home(request):
    return render(request,'app_scchen/home.html', response)

print('app_scchen was loaded!')

        # Saving data with other file format

        ## Save to text file
        # Save data
        f = open('pk_politician.txt','w')
        f.write(str(data_pk))
        f.close()

        # Load data
        f = open('pk_politician.txt','r')
        data_pk = f.read()

        # dictionary format
        eval(data_pk)
        ## Save to json file
        import json

        # Save data
        with open('pk_politician.txt', 'w') as file:
            json.dump( data_pk, file)

        # Load data
        with open('pk_politician.txt', 'r') as file:
            data_pk = json.load(file)

        # dictionary format
        data_pk

        ## save to mongoDb (A famous NoneSQL)
        from pymongo import MongoClient

        # Save data
        client = MongoClient()
        database = client["cnaNews"]  # SQL: Database Name
        table_KTH = database["pk_politicianTsaiHan"]   # SQL: Table Name

        table_KTH.drop()
        table_KTH.insert_one(data)


        # Load data
        from pymongo import MongoClient
        client = MongoClient()
        database = client["cnaNews"]  # SQL: Database Name
        table_KTH  = database["pkData"]   # SQL: Table Name

        for x in table_KTH.find():
            print(x)

        list(table_KTH.find({}, {'_id': False}))
